# IS434: Social Analytics & Apps
### SkillsFuture Industry Scraper
---

In [1]:
import pandas as pd
import os
from docx2python import docx2python
import json
pd.set_option('display.max_rows', None)

### Helper functions for Scraping

In [2]:
def clean_df( df ):
    # Clean first column
    df.iloc[:, 0] = df.iloc[:, 0].apply( lambda val : str(val[0]) )

    # Clean sector, track, subtrack, occupation, job role and JD
    for i in range(1, 8):
        df.at[i, 1] = [ elem for elem in df.at[i, 1] if elem != "" ]
        df.at[i, 1] = "\n\n".join(df.at[i, 1]) # only for job description, the rest are single element lists

    return df

In [3]:
def clean_unicode( string ):
    if "\u2018" in string:
        string = string.replace("\u2018", "\'")
    if "\u2019" in string:
        string = string.replace("\u2019", "\'")
    if "\u201c" in string:
        string = string.replace("\u201c", "\"")
    if "\u201d" in string:
        string = string.replace("\u201d", "\"")
    if "\u00a0" in string:
        string = string.replace("\u00a0", "")
    if "\u2013" in string:
        string = string.replace("\u2013", "-")
    return string

In [4]:
def scrape_job_tasks( df ):
    # JD info is between columns [Critical Work Functions + Key Tasks] and [Technical Skills and Competencies + Generic Skills and Competencies]
    # Step 1: Dynamically retrieve starting and ending index where heading [Critical Work Function] is
    start = 0
    end = 0

    for i in range( df.shape[0] ):
        value = df.at[i, 1]
        if value != "" and value != None:
            if "Critical Work Function" in value:
                start = i + 1 # because next line is where the first CWF is
            elif "Technical Skills and Competencies" in value:
                end = i # python stops 1 val before stated index, so no subtracting required

    # Step 2: Slice the dataframe into those with job_skills data only
    jd_df = df.iloc[start:end, 1:3]
    end = jd_df.shape[0]

    # Step 3: Go through the range and find the indexes where each Critical Work Function starts
    indexes_list = []
    for i in range( jd_df.shape[0] ):
        if jd_df.iloc[i, 0] != ['']:
            indexes_list.append( i )

    # Step 4: For each critical work function, append the key tasks into the dictionary 
    i = 0
    data_dict = {}
    
    for index in indexes_list:
        # 3
        critical_work_function = jd_df.iloc[index, 0]

        if type(critical_work_function) == str:
            critical_work_function = critical_work_function
        elif type(critical_work_function) == list:
            if len(critical_work_function) == 1:
                critical_work_function = critical_work_function[0]
            else:
                critical_work_function = "".join(critical_work_function)

        if index == indexes_list[-1]:
            # if it is the last element in the indexes_list, it should scrape till the end of the dataframe
            key_tasks = [ task[0] for task in jd_df.iloc[ index : end, 1] ]
        else:
            # else, scrape from the current row till one row before the next critical work function
            next_index = indexes_list[i+1]
            key_tasks = [ task[0] for task in jd_df.iloc[ index : next_index, 1] ]

        # Clean some unicode quotation marks from the code
        critical_work_function = clean_unicode( critical_work_function )
        key_tasks_cleaned = [ clean_unicode(task) for task in key_tasks ]
        data_dict[ critical_work_function ] = key_tasks_cleaned
        i += 1
    
    return data_dict

In [5]:
def scrape_job_skills( df ):
    # Step 1: Dynamically retrieve indexes where section [Skills and Competencies] starts and ends
    start = 0
    end = 0

    for i in range( df.shape[0] ):
        value = df.at[i, 0]
        if value != "" and value != None:
            if "skills and competencies" in value.lower():
                start = i + 1 # because next line is where the first CWF is
            elif "programme" in value.lower() and "listing" in value.lower():
                end = i # python stops 1 val before stated index, so no subtracting required

    # Step 2: Slice the dataframe into those with Job's Skills data only
    skills_df = df.iloc[start:end, 1:].reset_index(drop=True)
    skills_df.columns = ['technical_skill', 'tsc_level', 'generic_skill', 'gsc_level']

    # Step 3: Clean columns
    def clean_tsc_level( level ):
        # Some jobs have 2 levels e.g. Level 4, Level 5
        # We will only keep the lower requirement
        if len( level ) == len("Level X"):
            return "L" + level[0][-1]
        else:
            lower_level = level[0].split(",")
            return "L" + lower_level[0][-1]

    skills_df['tsc_level'] = skills_df['tsc_level'].apply( clean_tsc_level ) # convert from e.g. Level 4 -> L4
    skills_df['gsc_level'] = skills_df['gsc_level'].apply( lambda level : level[0] if level != None else "")
    skills_df['technical_skill'] = skills_df['technical_skill'].apply( lambda skill : skill[0] ) # remove list
    skills_df['generic_skill'] = skills_df['generic_skill'].apply( lambda skill : skill[0] )

    # Step 4: Scrape Technical Skills
    technical_dict = {}

    for _, row in skills_df[['technical_skill', 'tsc_level']].iterrows():
        technical_skill = clean_unicode( row['technical_skill'] )
        technical_dict[ technical_skill ] = row['tsc_level']

    # Step 5: Scrape Generic Skills
    generic_dict = {}
    for _, row in skills_df[['generic_skill', 'gsc_level']].iterrows():
        if row['generic_skill'] == "":
            break
        else:
            generic_skill = clean_unicode( row['generic_skill'] )
            generic_dict[ generic_skill ] = row['gsc_level']
    
    return technical_dict, generic_dict

### Scrape Job Description + Job Skills (Technical + Generic)

In [6]:
scraped_data = []

In [7]:
sector_dir_list = [ dir for dir in os.listdir("./") if "_Sector" in dir ]

job_id = 0

for sector_dir in sector_dir_list:
    # retrieve each sector's subfolder
    skill_dir_list = [ dir for dir in os.listdir(f"./{sector_dir}/") ]
    for skill_dir in skill_dir_list:
        # retrieve each .docx in a subfolder
        docx_files_list = [ docx for docx in os.listdir(f"./{sector_dir}/{skill_dir}/") if docx.endswith(".docx") ]

        # scrape data from each .docx file
        for doc_file in docx_files_list:
            # Load the file
            doc = docx2python(f"./{sector_dir}/{skill_dir}/{doc_file}")
            table = doc.body[0]
            df = clean_df( pd.DataFrame(table) )

            # Instantiate dictionary to store job's information
            # job_id += 1
            job_dict = {
                # "id": job_id,
                "sector": "",
                "track": "",
                "subtrack": "",
                "job_role": "",
                "description": "",
                "critical_work_functions" : [],
                "technical_sc": [],
                "generic_sc": []
            }

            # === PART A: Scrape Job Role and Industry Information ===
            for _, row in df.iterrows():
                heading = row[0]
                if heading == 'Sector':
                    job_dict['sector'] = row[1]
                elif heading == 'Track':
                    job_dict['track'] = row[1]
                elif heading == "Sub-track":
                    job_dict['subtrack'] = row[1]
                elif heading == 'Job Role':
                    job_dict['job_role'] = row[1]
                elif heading == 'Job Role Description':
                    job_dict['description'] = clean_unicode( row[1] )

            # === PART B: Scrape Job Descriptions ===
            job_tasks_dict = scrape_job_tasks( df )
            job_dict['critical_work_functions'] = job_tasks_dict

            # === PART C: Scrape Job Skills ===
            technical_skills_dict, generic_skills_dict = scrape_job_skills( df )
            job_dict['technical_sc'] = technical_skills_dict
            job_dict['generic_sc'] = generic_skills_dict

            scraped_data.append( job_dict )

### Write output to JSON

In [8]:
with open('jobs_info.json', 'w') as outfile:
    outfile.write( json.dumps( scraped_data, indent = 4) )

In [9]:
# with open('jobs_info_with_id.json', 'w') as outfile:
#     outfile.write( json.dumps( scraped_data, indent = 4) )